## Intro
This notebook provides the code used to compute some of the results in the paper, "Applications of Conformal Geometric Algebra to Transmission Line Theory". It makes use of the [galgebra](https://github.com/brombo/galgebra) python module.

In [2]:
from sympy import * 
from galgebra.ga import *
from galgebra.mv import * 


cf3g = '1 0 0 0, 0 1 0 0, 0 0 1 0, 0 0 0 -1'
cf3 = Ga('e_1 e_2 e_3 e_4', g = cf3g)
(e1, e2, e3, e4) = cf3.mv()

alpha1,alpha2,alpha3,alpha4,beta, theta, phi = symbols('alpha1 alpha2 alpha3 alpha4 beta theta phi')
half = Rational(1,2)

# setup  null basis, and minkowski subspace bivector
eo = half^(e4-e3) # use rational for 1/2
einf = e3+e4
E0 = einf^eo
I = e1^e2^e3^e4
e1234 = e1^e2^e3^e4

In [3]:
from sympy import I as i
alpha,beta = symbols('alpha beta', real=True)

up = lambda x: x + (half^((x*x)*einf)) + eo  # ! '**' doesnt work for scalars # !use half
homo = lambda x: x * (-x|einf).inv() # ! inv syntax
down = lambda x: (homo(x)^E0)*E0

c2v = lambda x: (re(x)*e1) + (im(x)*e2) # ! sympy has re(), im() not properties
v2c = lambda x: (x|e1)+ (x|e2)*i

cot = lambda x: 1./tan(x)

Rot = lambda V,X: V*X*V.inv()
t = lambda A: 1-(.25*(einf*A)*(einf|A).inv()*(1+E0))
T =lambda a: 1+(half^(einf*a))



A = cf3.mv('a','vector')
a = down(A)
assert (T(a)==t(up(a)))

In [49]:


## conformal operations
CI = lambda X: Rot((e2^e3),X) # complex inversion
Add = lambda X,Y: Rot(t(Y),X)
#Mul = lambda X,Y: 


## basis transforms
Rzy = exp(-pi/2*(e2^e3))
Ryz = ~Rzy
Rsz = exp(pi/4*(e1^e3))
Rzs = ~Rsz
Rsy = Rsz*Rzy
Rys = ~Rsy

### Transmission line in S,Y, and Z

In [59]:
Ls = exp(theta*e1*e2)
Lz = Rzs*Ls*Rsz
assert(Lz== exp(-theta*e2*e3))

In [60]:
Ly = Rys*Ls*Rsy
assert(Ly== exp(-theta*e2*e3))

### Bivector Algebra and Rotors

In [71]:
# symbols
r,x,g,b,theta,rho,q,n = symbols('r,x,g,b,theta,rho,q,n')

# bivector algebra 
R =  e3*e4-e1*e3
X = -e2*e4+e1*e2
G =  e3*e4+e1*e3
B =  e2*e4+e1*e2
N =  e1*e4
Q =  e3*e2

# others
L = e1^e2
A = E0

# Rotors
Rr = exp(half*r*R)
Rx = exp(half*x*X)
Rg = exp(half*g*G)
Rb = exp(half*b*B)
Rn = exp(-half*log(n)*N)
Rq = exp(half*q*Q)

# others
Rl = exp(theta*L)
Ra = exp(-half*log(rho)*A)

#test rotors do what we want
assert(Rsz*T(r*e1)*Rzs == Rr)
assert(Rsz*T(x*e2)*Rzs == Rx)
assert(Rsy*T(g*e1)*Rys == Rg)
assert(Rsy*T(b*e2)*Rys == Rb)


# duality relations 
assert(R*I == X)
assert(G*I == B)
assert(A*I == L)
assert(Q*I == N)

# time-reversal relation
assert( e4*X*e4 == -B)
assert( e4*R*e4 ==  G)
assert( e4*N*e4 ==  N)
assert( e4*Q*e4 == -Q)


# bivector classification
# light-like
assert(R**2==G**2==X**2==B**2==0)
#timelike
assert(N**2==A**2==1)
#spacelike
assert(Q**2==L**2==-1)


###  Bivector Commutator table

In [6]:
#diagonal 
for k in [R,X,G,B,N,Q]:
    assert(k>>k ==0)

assert(R>>G==-2*N)
assert(R>>X==0)
assert(R>>B==2*Q)
assert(R>>N==R)
assert(R>>Q==-X)

assert(G>>X==-2*Q)
assert(G>>B==0)
assert(G>>N==-G)
assert(G>>Q==B)

assert(X>>B ==2*N)
assert(X>>N ==X)
assert(X>>Q ==R)

assert(B>>N==-B)
assert(B>>Q==-G)

assert(N>>Q ==0)


## Impedance Transformer

Ensure an N-rotation is a scaling in z. 

In [8]:
down(Rot(Rzs*Rn*Rsz,up(alpha*e1)))

alpha*n*e_1

In [9]:
down(Rot(Rn,up(eo)))

tanh(log(n)/2)*e_1

Compute the rotor for a mismatched line

In [10]:
Rml = ~Rn*Rl*Rn
Rml

cos(theta) + sin(theta)*cosh(log(n))*e_1^e_2 + sin(theta)*sinh(log(n))*e_2^e_4

A line of 90deg inverts a real load's impedance

In [11]:
Rl90 = Rl.subs({'theta':pi/2})
down(Rot(Rzs*Rl90*Rsz,up(alpha*e1)))

e_1/alpha

The rotor for a mismatched line 

In [12]:
Rml = ~Rn*Rl*Rn
Rml

cos(theta) + sin(theta)*cosh(log(n))*e_1^e_2 + sin(theta)*sinh(log(n))*e_2^e_4

At 90deg this becomes

In [13]:

Rml90 = Rml.subs({'theta':pi/2})
Rml90

cosh(log(n))*e_1^e_2 + sinh(log(n))*e_2^e_4

which has the effect on a real load 

In [14]:
down(Rot(Rml90,up(alpha*e1 )))

-(alpha*cosh(log(n)) + sinh(log(n)))*e_1/(alpha*sinh(log(n)) + cosh(log(n)))

and for a match, this is 


In [15]:
down(Rot(~Rml90,up(0 )))

-tanh(log(n))*e_1

In [16]:
V=(Rn*Rq)
N*Q ==Q*N

True

In [17]:
Rot(Rn,eo)

sinh(log(n))*e_1/2 - e_3/2 + cosh(log(n))*e_4/2

In [18]:
Rq*Rn*~Rq*Rl*~Rq*~Rn*Rq

cos(theta) + sin(theta)*cosh(log(n))*e_1^e_2 - sin(theta)*sinh(log(n))*e_2^e_4

In [19]:
down(Rot(Rml,eo))

-sin(theta)**2*sinh(2*log(n))*e_1/(sin(theta)**2*cosh(2*log(n)) - sin(theta)**2 + 2) - sin(2*theta)*sinh(log(n))*e_2/(2*sin(theta)**2*sinh(log(n))**2 + 2)

In [20]:
down(Rot(Rzs*Rml90*Rsz,up(alpha*e1 )))

e_1/(alpha*n**2)

In [21]:
gamma_l = (down(~Rml*eo*Rml))
gamma_l**2


sin(theta)**4*sinh(log(n))**2*cosh(log(n))**2/(-sin(theta)**2*sinh(log(n))**2/2 + sin(theta)**2*sinh(log(n))**2/2 + sin(theta)**2*cosh(log(n))**2 + cos(theta)**2)**2 + sin(theta)**2*cos(theta)**2*sinh(log(n))**2/(-sin(theta)**2*sinh(log(n))**2/2 + sin(theta)**2*sinh(log(n))**2/2 + sin(theta)**2*cosh(log(n))**2 + cos(theta)**2)**2

sinh(log(n))*e_1/2 - e_3/2 + cosh(log(n))*e_4/2

In [22]:

(gamma_l|e2)/(gamma_l|e1)

-cos(theta)/(sin(theta)*cosh(log(n)))

The center of rotation of a mismatched line may be found by rotating the origina by the impedance scaling rotor, 

In [41]:
Rn*eo*~Rn


sinh(log(n))*e_1/2 - e_3/2 + cosh(log(n))*e_4/2

In [42]:
down(Rn*eo*~Rn)

tanh(log(n)/2)*e_1

In [46]:
x = sinh(x)+cosh(x)
b = cosh(x)-sinh(x)
x*b

(sinh(sinh(x) + cosh(x)) + cosh(sinh(x) + cosh(x)))*(-sinh(sinh(sinh(x) + cosh(x)) + cosh(sinh(x) + cosh(x))) + cosh(sinh(sinh(x) + cosh(x)) + cosh(sinh(x) + cosh(x))))

In [47]:
cosh(x)**2 - sinh(x)**2

-sinh(sinh(sinh(x) + cosh(x)) + cosh(sinh(x) + cosh(x)))**2 + cosh(sinh(sinh(x) + cosh(x)) + cosh(sinh(x) + cosh(x)))**2

###  Stub Matching

First find the susceptance of a shorted/opened stub.



In [24]:
short = up(-e1)
open = up(e1)

In [25]:

down((Rys*Rl*open*~Rl*Rsy))

tan(theta)*e_2

In [26]:
down((Rys*Rl*short*~Rl*Rsy))

-e_2/tan(theta)

Define rotations such stubs generate

In [27]:
Ros =exp(half*tan(phi)*B) #open stub
Rss =exp(half*-1/tan(phi)*B) #short stub
Rss

1 - e_1^e_2/(2*tan(phi)) - e_2^e_4/(2*tan(phi))

Next the matching problem is inverted. Assume a match at port1, looking in from port 2, we see series line `Rl`,  shunt stub `Rss`, a and matched load.

In [28]:
F= Rl*Ros
F

-sin(theta)*tan(phi)/2 + cos(theta) + (sin(theta) + cos(theta)*tan(phi)/2)*e_1^e_2 + sin(theta)*tan(phi)*e_1^e_4/2 + cos(theta)*tan(phi)*e_2^e_4/2

In [29]:
d = down(F*eo*~F)

d**2

(sin(theta)**2*tan(phi)/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) + sin(theta)*cos(theta)*tan(phi)**2/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - cos(theta)**2*tan(phi)/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)))**2 + (sin(theta)**2*tan(phi)**2/(4*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - sin(theta)*cos(theta)*tan(phi)/(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2) - cos(theta)**2*tan(phi)**2/(4*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)))**2

This is  complicated, but if we look at it in polar repr

In [30]:
gamma_l = down(F*eo*~F)
(gamma_l|e2)/(gamma_l|e1)



sin(theta)**2*tan(phi)/(sin(theta)**4*tan(phi)**4/(8*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) + sin(theta)**4*tan(phi)**2/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - sin(theta)**3*cos(theta)*tan(phi)**3/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - 2*sin(theta)**3*cos(theta)*tan(phi)/(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2) - sin(theta)*cos(theta)**3*tan(phi)**3/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - 2*sin(theta)*cos(theta)**3*tan(phi)/(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2) - cos(theta)**4*tan(phi)**4/(8*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - cos(theta)**4*tan(phi)**2/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 

In [31]:
gamma_l**2 #and mag

(sin(theta)**2*tan(phi)/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) + sin(theta)*cos(theta)*tan(phi)**2/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - cos(theta)**2*tan(phi)/(2*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)))**2 + (sin(theta)**2*tan(phi)**2/(4*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)) - sin(theta)*cos(theta)*tan(phi)/(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2) - cos(theta)**2*tan(phi)**2/(4*(sin(theta)**2*tan(phi)**2/4 + sin(theta)**2 + cos(theta)**2*tan(phi)**2/4 + cos(theta)**2)))**2